# Model Specification for 1st-Level fMRI Analysis

Nipype provides also an interfaces to create a first level Model for an fMRI analysis. Such a model is needed to specify the study specific information, such as **condition**, their **onsets** and **durations**. For more information, make sure to check out [Model Specificaton](http://nipype.readthedocs.io/en/latest/users/model_specification.html) and [nipype.algorithms.modelgen](http://nipype.readthedocs.io/en/latest/interfaces/generated/nipype.algorithms.modelgen.html)

## Simple Example

Let's consider a simple experiment, where we have three different stimuli such as ``'faces'``, ``'houses'`` and ``'scrambled pix'``. Now each of those three conditions has different stimuli onsets, but all of them have a stimuli presentation duration of 3 seconds.

So to summarize:

    conditions = ['faces', 'houses', 'scrambled pix']
    onsets = [[0, 30, 60, 90],
              [10, 40, 70, 100],
              [20, 50, 80, 110]]
    durations = [[3], [3], [3]]
    
The way we would create this model with Nipype is almsot as simple as that. The only step that is missing is to put this all into a ``Bunch`` object. This can be done as follows:

In [ ]:
from nipype.interfaces.base import Bunch

conditions = ['faces', 'houses', 'scrambled pix']
onsets = [[0, 30, 60, 90],
          [10, 40, 70, 100],
          [20, 50, 80, 110]]
durations = [[3], [3], [3]]

subject_info = Bunch(conditions=conditions,
                     onsets=onsets,
                     durations=durations)

It's also possible to specify additional regressors. For this you need to additionally specify:

- **``regressors``**: list of regressors that you want to include in the model (must correspond to the number of volumes in the functional run)
- **``regressor_names``**: name of the regressors that you want to include

## Example based on dataset

Now for a more realistic example, let's look at a TVA file from our tutorial dataset.

In [ ]:
!cat /data/ds102/sub-01/func/sub-01_task-flanker_run-1_events.tsv

onset	duration	trial_type	response_time	correctness	StimVar	Rsponse	Stimulus	cond
0.0	2.0	incongruent_correct	1.095	correct	2	1	incongruent	cond003
10.0	2.0	incongruent_correct	0.988	correct	2	1	incongruent	cond003
20.0	2.0	congruent_correct	0.591	correct	1	1	congruent	cond001
30.0	2.0	congruent_correct	0.499	correct	1	1	congruent	cond001
40.0	2.0	incongruent_correct	0.719	correct	2	1	incongruent	cond003
52.0	2.0	congruent_correct	0.544	correct	1	1	congruent	cond001
64.0	2.0	congruent_correct	0.436	correct	1	1	congruent	cond001
76.0	2.0	incongruent_correct	0.47	correct	2	1	incongruent	cond003
88.0	2.0	congruent_correct	0.409	correct	1	1	congruent	cond001
102.0	2.0	incongruent_correct	0.563	correct	2	1	incongruent	cond003
116.0	2.0	congruent_correct	0.493	correct	1	1	congruent	cond001
130.0	2.0	congruent_correct	0.398	correct	1	1	congruent	cond001
140.0	2.0	congruent_correct	0.466	correct	1	1	congruent	cond001
150.0	2.0	incongruent_correct	0.518	correct	2	1	incongruent	cond003
164.0	2.0

So, the only things that we need to specify our model are the onset and the stimuli type, i.e. **column 0** and **column 5 or 7**. Those we can get with the command:

In [ ]:
import numpy as np
filename = '/data/ds102/sub-01/func/sub-01_task-flanker_run-1_events.tsv'
trailinfo = np.genfromtxt(filename, delimiter='\t', dtype=None, skip_header=1)
trailinfo = [[t[0], t[7]] for t in trailinfo]
trailinfo

Before we can use the onsets, we first need to split them into the two conditions:

In [ ]:
onset1 = []
onset2 = []

for t in trailinfo:
    if 'incongruent' in t[1]:
        onset2.append(t[0])
    else:
        onset1.append(t[0])

print onset1
print onset2

The last thing we now need to to is to put this into a ``Bunch`` object and we're done:

In [ ]:
from nipype.interfaces.base import Bunch

conditions = ['congruent', 'incongruent']
onsets = [onset1, onset2]
durations = [[2], [2]]

subject_info = Bunch(conditions=conditions,
                     onsets=onsets,
                     durations=durations)